In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
#from peft import LoraConfig
#from trl import SFTTrainer

In [ ]:
!pip install transformers accelerate

# Login to Hugging Face

In [5]:
# LOGIN INTO HUGGINGFACE WITH TOKEN
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGGFACE_TOKEN")

!pip install huggingface_hub[inference]

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 12.5 MB/s eta 0:00:00 0:00:01


In [6]:
login(token=hf_token)

#model_name = "meta-llama/Llama-2-7b-hf"
# Load the tokenizer and model
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
#model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Using the LLAMA2 - CHAT - MODEL

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    # cache_dir="/data/yash/base_models",
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", 
                                          # cache_dir="/data/yash/base_models"
                                         )

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:

# function for model inference 
def get_llama2_chat_reponse(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,  temperature=1.1) #temperature= 0.00001)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Prompt 

In [10]:
caption_to_correct="a warning mark is required on the perimeter of the excavation in contact with vehicles warning lights "

In [9]:
def correct_prompt(caption_to_correct):
    prompt =f"<<SYS>>\
You are an engineer assistant specialized in industry, factories, industry safety.You will be given 1 sentence. \
The sentence describes a problem in the industry domain or it can also have no problem . \
The sentence may not be complete or may be very brief and will not always be grammatically or syntactically accurate . \
If the sentence describes a problem, understand the problem from the sentence and generate 3 sentences that could potentially solve the problem described. \
The sentences you provide must be solutions to the problem and be grammatically and syntactically accurate. Do not repeat the prompt i give you. \
Number each provided sentence. If you can not understand the sentence at all do not provide incorrect solutions. Do not provide dangerous solutions. \
\
If there is no problem in the sentence answer with 'no problem'. \
<</SYS>>\
[INST]Suggest 3 solutions to correct the problem in the sentence : fire extinguisher blocking door.[/INST]\
\Solution :\
There is a problem. The fire extinguisher prevents access to the door. This could be dangerous, so I will provide solutions. \
Solutions : \
1. Relocate the fire extinguisher: Consider repositioning the fire extinguisher to a location that does not obstruct the door, such as a nearby wall or corner. This would ensure that the fire extinguisher is easily accessible without blocking any doors.\
2. Install a sliding fire extinguisher: Design a fire extinguisher that can be easily slid out of the way when not in use, allowing the door to open freely. This could involve using a lightweight and compact design, or incorporating a sliding mechanism into the fire extinguisher's casing.\
3. Use a retractable fire extinguisher: Develop a fire extinguisher that can be retracted into a housing when not in use, allowing the door to open without obstruction. This could involve using a spring-loaded or motorized mechanism to retract the fire extinguisher into its housing.\
\
[INST]Suggest 3 solutions to correct the problem in the sentence : freezer compartment [/INST]\
There is no problem, since the caption describes only a freezer. Therefore, I will provide no solutions. \
\Solution : no problem \
\
[INST]Suggest 3 solutions to correct the problem in the sentence : car in parking [/INST]\
There is no problem, since the caption describes only a car in a parking. Therefore, I will provide no solutions. \
\Solution : no problem \
\
[INST]Suggest 3 solutions to correct the problem in the sentence : flowers rd [/INST]\
There is no problem, since the caption describes only flowers. Therefore, I will provide not solutions. \
\Solution : no problem \
\
[INST].Suggest 3 solutions to correct the problem in the sentence : {caption_to_correct}\
Solution : [/INST]"
    return prompt

In [11]:
# GET RESULTS 
correction_prompt = correct_prompt(caption_to_correct)
solutions = get_llama2_chat_reponse(correction_prompt, max_new_tokens=400)
result = solutions.replace(correction_prompt,'')

In [12]:
print(result)

There is a problem. The sentence describes a situation where a warning mark is required on the perimeter of an excavation to warn vehicles of potential danger. This could be dangerous, so I will provide solutions. Solutions : 1. Install warning lights: Place warning lights along the perimeter of the excavation to alert drivers of the potential hazard. These lights could be placed at regular intervals, such as every 10 feet, to ensure that drivers are aware of the danger.2. Use reflective barriers: Position reflective barriers along the perimeter of the excavation to create a visible warning to drivers. These barriers could be made of a reflective material, such as aluminum foil, and could be placed at the same intervals as the warning lights.3. Post warning signs: Place signs at the entrance to the excavation, clearly warning drivers of the potential danger. These signs could include a message such as "Caution: Excavation Ahead" or "Danger: Unexpected Hazard".


# Not bad  : The model can provide solutions but the problem is it provides solutions even in case there is no actual problem in the sentence. We need  to think of better prompts or other ideas to make the model do not answer with solutions if there is no issue detected.

- Solution : 
        - edit prompt to prevent solutions when there are no problems 
        - modify and increase the temperature to 1.1 close to 1 to make the answers more diverse and prevent the conservative "always suggest solution" approach

### This is similar to chain of thought prompting, but mainly we used few shot examples to steer the model towards the direction we want.
### Let's try with chain of thought process now 

In [51]:

# function for model inference 
def get_llama2_chat_reponse2(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,  temperature=0.8) #temperature= 0.00001)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [56]:
caption_to_correct2 = "service cable in escape route"

In [53]:
def correct_prompt2(caption_to_correct):
    prompt =f"<<SYS>>\
You are an engineer assistant specialized in industry, factories, industry safety.You will be given 1 sentence. \
The sentence describes a problem in the industry domain or it can also have no problem . \
The sentence may not be complete or may be very brief and will not always be grammatically or syntactically accurate . \
If the sentence describes a problem, understand the problem from the sentence and generate 3 sentences that could potentially solve the problem described. \
The sentences you provide must be solutions to the problem and be grammatically and syntactically accurate. Do not repeat the prompt i give you. \
Number each provided sentence. If you can not understand the sentence at all do not provide incorrect solutions. Do not provide dangerous solutions. \
\
If there is no problem in the sentence answer with 'no problem'. \
<</SYS>>\
[INST]Suggest 3 solutions to correct the problem in the sentence : fire extinguisher blocking door.[/INST]\
\Answer :\
Let's try to detect the problems in the caption. The caption talks about a fire extinguisher, which blocks a door. From this I undestand that there is a problem. The fire extinguisher prevents access to the door. This could be dangerous, so I will provide solutions. \
Solutions : \\\n \
1. Relocate the fire extinguisher: Consider repositioning the fire extinguisher to a location that does not obstruct the door, such as a nearby wall or corner. This would ensure that the fire extinguisher is easily accessible without blocking any doors.\\\n \
2. Install a sliding fire extinguisher: Design a fire extinguisher that can be easily slid out of the way when not in use, allowing the door to open freely. This could involve using a lightweight and compact design, or incorporating a sliding mechanism into the fire extinguisher's casing.\\\n \
3. Use a retractable fire extinguisher: Develop a fire extinguisher that can be retracted into a housing when not in use, allowing the door to open without obstruction. This could involve using a spring-loaded or motorized mechanism to retract the fire extinguisher into its housing.\\\n \
\
[INST]Suggest 3 solutions to correct the problem in the sentence : freezer compartment [/INST]\
\Answer :\
Let's try to detect the problems in the caption. There is no problem, since the caption describes only a freezer compartement. A freezer compartement is only an object so there is no danger here. Therefore, I will provide no solutions. \
\Solution : no problem \
\
[INST].Suggest 3 solutions to correct the problem in the sentence : {caption_to_correct}\
Answer : [/INST]"
    return prompt

In [57]:
# GET RESULTS 
correction_prompt2 = correct_prompt2(caption_to_correct2)
solutions2 = get_llama2_chat_reponse2(correction_prompt2, max_new_tokens=400)
result2 = solutions2.replace(correction_prompt2,'')

In [58]:
print(result2)

  Sure, I'm happy to help! Based on the sentence "service cable in escape route," I understand that there is a problem. The service cable is located in the escape route, which could pose a danger to people trying to evacuate the building in case of an emergency. Here are three possible solutions to address this problem: \

1. Reroute the service cable: Identify an alternative route for the service cable that avoids the escape route. This could involve reconfiguring the building's infrastructure or installing a new cable that bypasses the escape route. \
2. Install a protective cover: Design a protective cover for the service cable that can be easily removed in case of an emergency. This would allow people to access the escape route without risking damage to the cable. \
3. Use a detachable service cable: Develop a service cable that can be easily detached from the main system in case of an emergency. This would allow people to access the escape route without having to worry about the c

# Great ! Now let's try to correct the new lines and some artifacts being printed in the output. We need to fix the format of our captions 
- We also need to add a case of "sentence which makes no sense at all" (only symbols or spaces). In this case the model should anwser "I don't undestand, please provide a caption i can understand"

In [80]:
caption_to_correct3 = "wooden placks on road"


In [63]:
# GET RESULTS 
correction_prompt2 = correct_prompt2(caption_to_correct3)
solutions2 = get_llama2_chat_reponse2(correction_prompt2, max_new_tokens=400)
result3 = solutions2.replace(correction_prompt2,'')

In [64]:
result3

'  I apologize, but I\'m not able to understand the problem in the sentence "sr df ps floor" as it is not a complete or clear sentence. Could you please provide more context or clarify the sentence? I\'ll be happy to help you generate solutions once I understand the problem better.'

# Let's now fix the format of the output 

In [81]:
def correct_prompt3(caption_to_correct):
    prompt =f"<<SYS>>\
You are an engineer assistant specialized in industry, factories, industry safety.You will be given 1 sentence. \
The sentence describes a problem in the industry domain or it can also have no problem . \
The sentence may not be complete or may be very brief and will not always be grammatically or syntactically accurate . \
If the sentence describes a problem, understand the problem from the sentence and generate 3 sentences that could potentially solve the problem described. \
The sentences you provide must be solutions to the problem and be grammatically and syntactically accurate. Do not repeat the prompt i give you. \
Number each provided sentence. If you can not understand the sentence at all do not provide incorrect solutions. Do not provide dangerous solutions. \
\
If there is no problem in the sentence answer with 'no problem'. \
<</SYS>>\
[INST]Suggest 3 solutions to correct the problem in the sentence : fire extinguisher blocking door.[/INST]\
\Answer :\
Let's try to detect the problems in the caption. The caption talks about a fire extinguisher, which blocks a door. From this I undestand that there is a problem. The fire extinguisher prevents access to the door. This could be dangerous, so I will provide solutions. \
Solutions :\
1. Relocate the fire extinguisher: Consider repositioning the fire extinguisher to a location that does not obstruct the door, such as a nearby wall or corner. This would ensure that the fire extinguisher is easily accessible without blocking any doors. \
2. Install a sliding fire extinguisher: Design a fire extinguisher that can be easily slid out of the way when not in use, allowing the door to open freely. This could involve using a lightweight and compact design, or incorporating a sliding mechanism into the fire extinguisher's casing. \
3. Use a retractable fire extinguisher: Develop a fire extinguisher that can be retracted into a housing when not in use, allowing the door to open without obstruction. This could involve using a spring-loaded or motorized mechanism to retract the fire extinguisher into its housing. \
\
[INST]Suggest 3 solutions to correct the problem in the sentence : freezer compartment [/INST]\
\Answer :\
Let's try to detect the problems in the caption. There is no problem, since the caption describes only a freezer compartement. A freezer compartement is only an object so there is no danger here. Therefore, I will provide no solutions. \
\Solution : no problem \
\
[INST].Suggest 3 solutions to correct the problem in the sentence : {caption_to_correct}\
Answer : [/INST]"
    return prompt

In [82]:
# GET RESULTS 
correction_prompt3 = correct_prompt3(caption_to_correct3)
solutions3 = get_llama2_chat_reponse2(correction_prompt3, max_new_tokens=400)
result3 = solutions3.replace(correction_prompt3,'')
result4 = result3.replace('\\','')
# maybe add postprocessing to drop all symbols from the anwser in the end

In [83]:
print(result4)

  Sure, I'd be happy to help! Let's try to detect the problem in the sentence. The sentence describes wooden planks on a road, which could potentially cause a problem. From this, I understand that there might be a danger or obstruction on the road. Here are three possible solutions:

1. Remove the wooden planks: The most straightforward solution would be to remove the wooden planks from the road. This would ensure that traffic can flow freely and safely without any obstruction.
2. Install a detour: If the wooden planks are too large or numerous to remove, a detour could be created to redirect traffic around the area. This would ensure that traffic can continue to flow without any interruptions or delays.
3. Secure the wooden planks: If the wooden planks are too large or heavy to remove, they could be secured in place using anchors or weights to prevent them from shifting or falling onto the road. This would ensure that the road remains safe for traffic even with the wooden planks prese